## Setup

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
base_dir = '/content/drive/MyDrive/ai_project'

In [ ]:
%cd {base_dir}

/content/drive/MyDrive/ai_project


In [ ]:
!apt-get install \
    git \
    cmake \
    ninja-build \
    build-essential \
    libboost-program-options-dev \
    libboost-graph-dev \
    libboost-system-dev \
    libeigen3-dev \
    libflann-dev \
    libfreeimage-dev \
    libmetis-dev \
    libgoogle-glog-dev \
    libgtest-dev \
    libgmock-dev \
    libsqlite3-dev \
    libglew-dev \
    qtbase5-dev \
    libqt5opengl5-dev \
    libcgal-dev \
    libceres-dev

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
build-essential is already the newest version (12.9ubuntu3).
libboost-program-options-dev is already the newest version (1.74.0.3ubuntu7).
libboost-program-options-dev set to manually installed.
libboost-system-dev is already the newest version (1.74.0.3ubuntu7).
libboost-system-dev set to manually installed.
libboost-graph-dev is already the newest version (1.74.0.3ubuntu7).
libboost-graph-dev set to manually installed.
cmake is already the newest version (3.22.1-1ubuntu1.22.04.2).
git is already the newest version (1:2.34.1-1ubuntu1.12).
libsqlite3-dev is already the newest version (3.37.2-2ubuntu0.4).
libsqlite3-dev set to manually installed.
The following additional packages will be installed:
  googletest libamd2 libbtf1 libcamd2 libccolamd2 libceres2 libcholmod3
  libcolamd2 libcxsparse3 libegl-dev libevdev2 libflann1.9 libfreeimage3
  libgflags-dev libgflags2.2 libgl-dev libglu1-mesa

In [ ]:
!apt update
!apt install -y colmap

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,721 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,736 kB]
Get:13 http://security.ubuntu.com/ubuntu jammy-s

## Extract keyframes

In [ ]:
import cv2
import numpy as np
import os

# PARAMETERS
VIDEO_PATH = "3D_reconstruction/200S0264.mp4"
OUTPUT_DIR = "video_4/keyframes"
MASK_PATH = "video_4/mask/mask.png"
MIN_FEATURES = 2000
MIN_MOVEMENT = 20
FRAME_SKIP = 5

os.makedirs(OUTPUT_DIR, exist_ok=True)

# Load mask
mask = cv2.imread(MASK_PATH, cv2.IMREAD_GRAYSCALE)
if mask is None:
    raise FileNotFoundError(f"Could not load mask at {MASK_PATH}")
mask = (mask > 128).astype(np.uint8) * 255  # ensure binary

# Init video and SIFT
cap = cv2.VideoCapture(VIDEO_PATH)
sift = cv2.SIFT_create()
bf = cv2.BFMatcher(cv2.NORM_L2, crossCheck=True)

frame_id = 0
keyframe_id = 0
last_kp, last_desc = None, None

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    if frame_id % FRAME_SKIP != 0:
        frame_id += 1
        continue

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    if mask.shape != gray.shape:
        raise ValueError("Mask and video frame must have the same resolution.")

    kp, desc = sift.detectAndCompute(gray, mask)

    if desc is None or len(kp) < MIN_FEATURES:
        frame_id += 1
        continue

    if last_kp is None:
        # Save first frame
        cv2.imwrite(f"{OUTPUT_DIR}/keyframe_{keyframe_id:04d}.jpg", frame)
        last_kp, last_desc = kp, desc
        keyframe_id += 1
    else:
        matches = bf.match(desc, last_desc)
        if len(matches) == 0:
            frame_id += 1
            continue

        movement = np.mean([
            np.linalg.norm(np.array(kp[m.queryIdx].pt) - np.array(last_kp[m.trainIdx].pt))
            for m in matches
        ])

        if movement > MIN_MOVEMENT:
            cv2.imwrite(f"{OUTPUT_DIR}/keyframe_{keyframe_id:04d}.jpg", frame)
            last_kp, last_desc = kp, desc
            keyframe_id += 1

    frame_id += 1

cap.release()
print(f"Saved {keyframe_id} keyframes to {OUTPUT_DIR}")

## Paths

In [ ]:
workspace_path = f"/{base_dir}/3D_reconstruction/200S0264.mp4"
image_path = f"{base_dir}/video_5_frames_preprocessed"
mask_path = f"{base_dir}/video_5_mask/mask.png"
db_path = f"{workspace_path}/database.db"
sparse_path = f"{workspace_path}/sparse"
dense_path = f"{workspace_path}/dense"

In [ ]:
import os

for path in [workspace_path, sparse_path, dense_path]:
  os.makedirs(path, exist_ok=True)

## Sparse reconstruction

In [ ]:
!colmap feature_extractor --help

COLMAP 3.7 (Commit Unknown on Unknown without CUDA)

Options can either be specified via command-line or by defining
them in a .ini project file passed to `--project_path`.

  -h [ --help ] 
  --random_seed arg (=0)
  --log_to_stderr arg (=0)
  --log_level arg (=2)
  --project_path arg
  --database_path arg
  --image_path arg
  --camera_mode arg (=-1)
  --image_list_path arg
  --descriptor_normalization arg (=l1_root)
                                        {'l1_root', 'l2'}
  --ImageReader.mask_path arg
  --ImageReader.camera_model arg (=SIMPLE_RADIAL)
  --ImageReader.single_camera arg (=0)
  --ImageReader.single_camera_per_folder arg (=0)
  --ImageReader.single_camera_per_image arg (=0)
  --ImageReader.existing_camera_id arg (=-1)
  --ImageReader.camera_params arg
  --ImageReader.default_focal_length_factor arg (=1.2)
  --ImageReader.camera_mask_path arg
  --SiftExtraction.num_threads arg (=-1)
  --SiftExtraction.use_gpu arg (=1)
  --SiftExtraction.gpu_index arg (=-1)
  --SiftExtract

In [ ]:
!colmap feature_extractor \
    --database_path {db_path} \
    --image_path {image_path} \
    --ImageReader.camera_mask_path {mask_path} \
    --ImageReader.single_camera 1 \
    --SiftExtraction.use_gpu 0


Feature extraction

Processed file [1/37]
  Name:            frame_5720.jpg
  Dimensions:      1920 x 1080
  Camera:          #1 - SIMPLE_RADIAL
  Focal Length:    2304.00px
  Features:        6913
Processed file [2/37]
  Name:            frame_5710.jpg
  Dimensions:      1920 x 1080
  Camera:          #1 - SIMPLE_RADIAL
  Focal Length:    2304.00px
  Features:        10153
Processed file [3/37]
  Name:            frame_5730.jpg
  Dimensions:      1920 x 1080
  Camera:          #1 - SIMPLE_RADIAL
  Focal Length:    2304.00px
  Features:        2731
Processed file [4/37]
  Name:            frame_5740.jpg
  Dimensions:      1920 x 1080
  Camera:          #1 - SIMPLE_RADIAL
  Focal Length:    2304.00px
  Features:        1294
Processed file [5/37]
  Name:            frame_5750.jpg
  Dimensions:      1920 x 1080
  Camera:          #1 - SIMPLE_RADIAL
  Focal Length:    2304.00px
  Features:        1202
Processed file [6/37]
  Name:            frame_5760.jpg
  Dimensions:      1920 x 1080
 

In [ ]:
!colmap sequential_matcher \
    --database_path {db_path} \
    --SiftMatching.use_gpu 0


Sequential feature matching

Matching image [1/37] in 8.969s
Matching image [2/37] in 5.748s
Matching image [3/37] in 2.291s
Matching image [4/37] in 1.673s
Matching image [5/37] in 1.530s
Matching image [6/37] in 1.429s
Matching image [7/37] in 2.580s
Matching image [8/37] in 2.166s
Matching image [9/37] in 1.328s
Matching image [10/37] in 1.393s
Matching image [11/37] in 1.242s
Matching image [12/37] in 1.454s
Matching image [13/37] in 1.478s
Matching image [14/37] in 1.501s
Matching image [15/37] in 1.604s
Matching image [16/37] in 3.183s
Matching image [17/37] in 3.063s
Matching image [18/37] in 2.028s
Matching image [19/37] in 1.969s
Matching image [20/37] in 2.267s
Matching image [21/37] in 2.355s
Matching image [22/37] in 3.117s
Matching image [23/37] in 3.648s
Matching image [24/37] in 2.222s
Matching image [25/37] in 2.283s
Matching image [26/37] in 2.402s
Matching image [27/37] in 2.389s
Matching image [28/37] in 5.149s
Matching image [29/37] in 4.497s
Matching image [30/37]

In [ ]:
!colmap mapper --help

COLMAP 3.7 (Commit Unknown on Unknown without CUDA)

Options can either be specified via command-line or by defining
them in a .ini project file passed to `--project_path`.

  -h [ --help ] 
  --random_seed arg (=0)
  --log_to_stderr arg (=0)
  --log_level arg (=2)
  --project_path arg
  --database_path arg
  --image_path arg
  --input_path arg
  --output_path arg
  --image_list_path arg
  --Mapper.min_num_matches arg (=15)
  --Mapper.ignore_watermarks arg (=0)
  --Mapper.multiple_models arg (=1)
  --Mapper.max_num_models arg (=50)
  --Mapper.max_model_overlap arg (=20)
  --Mapper.min_model_size arg (=10)
  --Mapper.init_image_id1 arg (=-1)
  --Mapper.init_image_id2 arg (=-1)
  --Mapper.init_num_trials arg (=200)
  --Mapper.extract_colors arg (=1)
  --Mapper.num_threads arg (=-1)
  --Mapper.min_focal_length_ratio arg (=0.10000000000000001)
  --Mapper.max_focal_length_ratio arg (=10)
  --Mapper.max_extra_param arg (=1)
  --Mapper.ba_refine_focal_length arg (=1)
  --Mapper.ba_refine_prin

In [ ]:
!colmap mapper \
    --database_path {db_path} \
    --image_path {image_path} \
    --output_path {sparse_path} \
    --Mapper.init_max_forward_motion 0.99 \
    --Mapper.init_min_tri_angle 0.5



Loading database

Loading cameras... 1 in 0.000s
Loading matches... 309 in 0.006s
Loading images... 37 in 0.037s (connected 37)
Building correspondence graph... in 0.021s (ignored 0)

Elapsed time: 0.001 [minutes]


Finding good initial image pair


Initializing with image pair #32 and #34


Global bundle adjustment

iter      cost      cost_change  |gradient|   |step|    tr_ratio  tr_radius  ls_iter  iter_time  total_time
   0  1.239159e+02    0.00e+00    4.70e+03   0.00e+00   0.00e+00  1.00e+04        0    2.25e-03    1.91e-02
   1  1.187091e+02    5.21e+00    5.77e+01   8.84e-01   9.97e-01  3.00e+04        1    1.15e-02    3.06e-02
   2  1.186794e+02    2.97e-02    2.00e+01   1.50e+00   9.98e-01  9.00e+04        1    2.43e-03    3.31e-02
   3  1.186566e+02    2.27e-02    1.40e+02   3.89e+00   7.50e-01  1.03e+05        1    2.36e-03    3.55e-02
   4  1.186307e+02    2.59e-02    1.11e+02   3.71e+00   8.43e-01  1.52e+05        1    2.65e-03    3.82e-02
   5  1.186109e+02    1.98e-02  

In [ ]:
!colmap model_converter \
    --input_path {sparse_path}/0 \
    --output_path {sparse_path}/model.ply \
    --output_type PLY

In [ ]:
from google.colab import files
files.download(f"{sparse_path}/model.ply")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import shutil
shutil.make_archive(workspace_path, 'zip', base_dir)

KeyboardInterrupt: 

## Dense reconstruction

### Build Colmap with CUDA support

In [ ]:
%cd /content/drive/MyDrive/

/content/drive/MyDrive


In [ ]:
!apt-get install \
    git \
    cmake \
    ninja-build \
    build-essential \
    libboost-program-options-dev \
    libboost-filesystem-dev \
    libboost-graph-dev \
    libboost-system-dev \
    libboost-test-dev \
    libeigen3-dev \
    libflann-dev \
    libfreeimage-dev \
    libmetis-dev \
    libgoogle-glog-dev \
    libgflags-dev \
    libsqlite3-dev \
    libglew-dev \
    qtbase5-dev \
    libqt5opengl5-dev \
    libcgal-dev \
    libceres-dev

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
build-essential is already the newest version (12.9ubuntu3).
libboost-filesystem-dev is already the newest version (1.74.0.3ubuntu7).
libboost-filesystem-dev set to manually installed.
libboost-program-options-dev is already the newest version (1.74.0.3ubuntu7).
libboost-program-options-dev set to manually installed.
libboost-system-dev is already the newest version (1.74.0.3ubuntu7).
libboost-system-dev set to manually installed.
libboost-graph-dev is already the newest version (1.74.0.3ubuntu7).
libboost-graph-dev set to manually installed.
libboost-test-dev is already the newest version (1.74.0.3ubuntu7).
libboost-test-dev set to manually installed.
cmake is already the newest version (3.22.1-1ubuntu1.22.04.2).
git is already the newest version (1:2.34.1-1ubuntu1.12).
libsqlite3-dev is already the newest version (3.37.2-2ubuntu0.3).
libsqlite3-dev set to manually installed.
The following

In [ ]:
!git clone https://github.com/colmap/colmap.git
%cd colmap
!mkdir build
%cd build
!cmake .. -GNinja
!ninja
!ninja install

fatal: destination path 'colmap' already exists and is not an empty directory.
/content/drive/MyDrive/colmap
mkdir: cannot create directory ‘build’: File exists
/content/drive/MyDrive/colmap/build
-- Enabling LSD support
-- Found FreeImage
--   Includes : /usr/include
--   Libraries : /usr/lib/x86_64-linux-gnu/libfreeimage.so
-- Found FLANN
--   Includes : /usr/include
--   Libraries : /usr/lib/x86_64-linux-gnu/libflann.so
-- Found LZ4
--   Includes : /usr/include
--   Libraries : /usr/lib/x86_64-linux-gnu/liblz4.so
-- Found Metis
--   Includes : /usr/include
--   Libraries : /usr/lib/x86_64-linux-gnu/libmetis.so
-- Found Glog
--   Target : glog::glog
-- Found Glew
--   Includes : /usr/include
--   Libraries : /usr/lib/x86_64-linux-gnu/libGLEW.so
-- Found required Ceres dependency: Eigen version 3.4.0 in /usr/include/eigen3
-- Found required Ceres dependency: glog
-- Found required Ceres dependency: gflags
-- Found Ceres version: 2.0.0 installed in: /usr with components: [EigenSparse, 

In [ ]:
%cd colmap/build

In [ ]:
!ninja install

[0/1] Install the project...
-- Install configuration: "Release"
-- Installing: /usr/local/share/applications/COLMAP.desktop
-- Installing: /usr/local/lib/libcolmap_controllers.a
-- Installing: /usr/local/lib/libcolmap_estimators.a
-- Installing: /usr/local/lib/libcolmap_exe.a
-- Installing: /usr/local/lib/libcolmap_feature_types.a
-- Installing: /usr/local/lib/libcolmap_feature.a
-- Installing: /usr/local/lib/libcolmap_geometry.a
-- Installing: /usr/local/lib/libcolmap_image.a
-- Installing: /usr/local/lib/libcolmap_math.a
-- Installing: /usr/local/lib/libcolmap_mvs.a
-- Installing: /usr/local/lib/libcolmap_optim.a
-- Installing: /usr/local/lib/libcolmap_retrieval.a
-- Installing: /usr/local/lib/libcolmap_scene.a
-- Installing: /usr/local/lib/libcolmap_sensor.a
-- Installing: /usr/local/lib/libcolmap_sfm.a
-- Installing: /usr/local/lib/libcolmap_util.a
-- Installing: /usr/local/lib/libcolmap_poisson_recon.a
-- Installing: /usr/local/lib/libcolmap_vlfeat.a
-- Installing: /usr/local/lib

### Dense reconstruction

In [ ]:
import subprocess

def run_command(command):
    process = subprocess.Popen(
        command,
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
        text=True
    )
    for line in process.stdout:
        print(line, end="")
    process.wait()

In [ ]:
run_command([
    "colmap", "image_undistorter",
    "--image_path", image_path,
    "--input_path", f"{sparse_path}/0",
    "--output_path", workspace_path,
    "--output_type", "COLMAP",
])


Reading reconstruction

 => Reconstruction with 37 images and 3779 points

Image undistortion

Undistorting image [1/37]
Undistorting image [2/37]
Undistorting image [3/37]
Undistorting image [4/37]
Undistorting image [5/37]
Undistorting image [6/37]
Undistorting image [7/37]
Undistorting image [8/37]
Undistorting image [9/37]
Undistorting image [10/37]
Undistorting image [11/37]
Undistorting image [12/37]
Undistorting image [13/37]
Undistorting image [14/37]
Undistorting image [15/37]
Undistorting image [16/37]
Undistorting image [17/37]
Undistorting image [18/37]
Undistorting image [19/37]
Undistorting image [20/37]
Undistorting image [21/37]
Undistorting image [22/37]
Undistorting image [23/37]
Undistorting image [24/37]
Undistorting image [25/37]
Undistorting image [26/37]
Undistorting image [27/37]
Undistorting image [28/37]
Undistorting image [29/37]
Undistorting image [30/37]
Undistorting image [31/37]
Undistorting image [32/37]
Undistorting image [33/37]
Undistorting image [34

In [ ]:
run_command([
    "colmap", "patch_match_stereo",
    "--workspace_path", workspace_path,
    "--workspace_format", "COLMAP",
    "--PatchMatchStereo.geom_consistency", "true"
])

In [ ]:
run_command([
    "colmap", "stereo_fusion",
    "--workspace_path", workspace_path,
    "--workspace_format", "COLMAP",
    "--input_type", "geometric",
    "--output_path", f"{dense_path}/fused.ply"
])


In [ ]:
run_command([
    "colmap", "poisson_mesher",
    "--input_path", f"{dense_path}/fused.ply",
    "--output_path", f"{dense_path}/meshed.ply",
])